In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
import cv2
from PIL import Image
from os import listdir

In [2]:
model_trained = load_model("D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/LabelledRice/rice_disease_classifer_v1_test_6676.model")
model_trained

In [ ]:
# import cv2
# # Load the input image from disk, clone it, and grab the image spatial dimensions
# path = 'D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Labelled/BrownSpot/brownspot1.png'
# image = cv2.imread(path)
# orig = image.copy()
# (h, w) = image.shape[:2]

In [37]:
# reshape
def reshape(path):
    basewidth = 300
    img = Image.open(path)
    wpercent = (basewidth/float(img.size[0]))
    #hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,basewidth), Image.ANTIALIAS)
    newpath = path.replace("Labelled", "Reshape")
    print(newpath)
    img.save(newpath)


In [ ]:
#background remove
from skimage.morphology import binary_closing, binary_opening, erosion

In [ ]:
selem = np.zeros((25, 25))

ci,cj=12, 12
cr=13

# Create index arrays to z
I,J=np.meshgrid(np.arange(selem.shape[0]),np.arange(selem.shape[1]))

# calculate distance of all points to centre
dist=np.sqrt((I-ci)**2+(J-cj)**2)

# Assign value of 1 to those points where dist<cr:
selem[np.where(dist<=cr)]=1

In [ ]:
%matplotlib inline
import numpy as np
from scipy import ndimage

# fig, ax = plt.subplots(20,2, figsize=(10,80))
idx = 0

bg_frac = 0
thres = 220
img = Image.open('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Reshape/BrownSpot/brownspot1.png')
im_arr = np.array(img)
#     ax[idx, 0].imshow(im_arr)
R = im_arr[:, :, 0]
G = im_arr[:, :, 1]
B = im_arr[:, :, 2]
while bg_frac < 0.6: 
    bg_mask = ((R>thres) | (B>thres))# & (G < 100)
    bg_frac = bg_mask.sum()/len(bg_mask.flatten())
    thres -= 5
# we use opening first since our mask is reversed (the foreground and background are reversed here)
bg_mask = binary_closing(erosion(binary_opening(bg_mask, selem), np.ones((3, 3))), np.ones((5,5)))

#Get biggest blob
label, num_label = ndimage.label(~bg_mask)
size = np.bincount(label.ravel())
biggest_label = size[1:].argmax() + 1
bg_mask = label == biggest_label

im_arr[~bg_mask, 0] = 255
im_arr[~bg_mask, 1] = 255
im_arr[~bg_mask, 2] = 255

img = Image.fromarray(im_arr)
img.save('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/BGRemoved/BrownSpot/brownspot1.png')
idx+=1

In [ ]:
'''
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import sys
from  PIL  import Image

img = cv.imread('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Reshape/BrownSpot/brownspot1.png', cv.IMREAD_UNCHANGED)
original = img.copy()

l = int(max(5, 6))
u = int(min(6, 6))

ed = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
edges = cv.GaussianBlur(img, (21, 51), 3)
edges = cv.cvtColor(edges, cv.COLOR_BGR2GRAY)
edges = cv.Canny(edges, l, u)

_, thresh = cv.threshold(edges, 0, 255, cv.THRESH_BINARY  + cv.THRESH_OTSU)
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
mask = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=4)

data = mask.tolist()
sys.setrecursionlimit(10**8)
for i in  range(len(data)):
    for j in  range(len(data[i])):
        if data[i][j] !=  255:
            data[i][j] =  -1
        else:
            break
    for j in  range(len(data[i])-1, -1, -1):
        if data[i][j] !=  255:
            data[i][j] =  -1
        else:
            break
image = np.array(data)
image[image !=  -1] =  255
image[image ==  -1] =  0

mask = np.array(image, np.uint8)

result = cv.bitwise_and(original, original, mask=mask)
result[mask ==  0] =  255
cv.imwrite('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/BGRemoved/BrownSpot/brownspot1.png', result)


img = Image.open('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/BGRemoved/BrownSpot/brownspot1.png')
img.convert("RGBA")
datas = img.getdata()

newData = []
for item in datas:
    if item[0] ==  255  and item[1] ==  255  and item[2] ==  255:
        newData.append((255, 255, 255, 0))
    else:
        newData.append(item)

img.putdata(newData)
img.save('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/BGRemoved/BrownSpot/brownspot1.png', "PNG")

''';

In [5]:
EPOCHS = 30
INIT_LR = 1e-3
BS = 16
default_image_size = tuple((256, 256))
image_size = 0
directory_root = 'D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/BGRemoved/BrownSpot'
width=256
height=256
depth=3

In [6]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [3]:
model2 = load_model("D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/LabelledRice/rice_disease_classifer_v1_test_6676.model")

In [15]:
an_image = convert_image_to_array('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Labelled/BrownSpot/brownspot1.jpg')
#print(an_image.shape)
model2.predict(an_image[np.newaxis])

array([[1., 0., 0., 0.]], dtype=float32)

In [11]:
an_image1 = convert_image_to_array('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Labelled/BrownSpot/brownspot2.png')
#print(an_image.shape)
model2.predict(an_image1[np.newaxis])

array([[1., 0., 0., 0.]], dtype=float32)

In [16]:
an_image2 = convert_image_to_array('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Labelled/Hispa/hispa1.jpg')
#print(an_image.shape)
model2.predict(an_image2[np.newaxis])

array([[1., 0., 0., 0.]], dtype=float32)

In [34]:
reshape('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Labelled/LeafBlast/leafblast1.png')
an_image3 = convert_image_to_array('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Reshape/LeafBlast/leafblast1.png')

#print(an_image.shape)
model2.predict(an_image3[np.newaxis])


D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Labelled/LeafBlast/leafblast1.png


array([[0., 0., 0., 1.]], dtype=float32)

In [48]:
reshape('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Labelled/Healthy/healthy1.png')
an_image3 = convert_image_to_array('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Reshape/Healthy/healthy1.png')

#print(an_image.shape)
model2.predict(an_image3[np.newaxis])

D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Reshape/Healthy/healthy1.png


array([[1., 0., 0., 0.]], dtype=float32)

In [51]:
#reshape('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Labelled/Healthy/healthy1.png')
an_image4 = convert_image_to_array('D://Moy Hao Zhen/University of Malaya/Sem 5 - Internship Intel/UMHackathon/archive/Test/Reshape/Healthy/healthy2.jpg')

#print(an_image.shape)
model2.predict(an_image4[np.newaxis])

array([[1., 0., 0., 0.]], dtype=float32)